In [ ]:
import pandas as pd
data=pd.read_csv('fake_or_real_news.csv')
trans_data=data.T.iloc[3]
new_data=data.loc[:,'title':'label']
list1=list(trans_data)
list2=[]
list3=[]
for i,j in enumerate(list1):
    if(j=='REAL' or j=='FAKE'):
        list2.append(i)
for i in list2:
        list3.append(new_data.iloc[i])
df=pd.DataFrame(list3)
df.replace(to_replace=['REAL','FAKE'],value=[1,0],inplace=True)
df.to_csv(r'data.csv')

In [ ]:
import pandas as pd
data=pd.read_csv('data.csv')

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

for i in range(0,len(data)):
    news=re.sub('[^a-zA-Z]',' ',data['text'][i])
    news=news.lower()
    news=news.split()
    news=[ps.stem(word) for word in news if not word in set(stopwords.words('english'))]
    news=' '.join(news)
    corpus.append(news)

corpus

df=pd.DataFrame(corpus)
df.to_csv('stem.csv')

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=42300)
x=cv.fit_transform(corpus).toarray()

x.shape

x

y=df.iloc[:,-1].values

from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.20, random_state=0)

In [ ]:
##naive Bayes
from sklearn.naive_bayes import GaussianNB
classifier =  GaussianNB()
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

from sklearn.metrics import confusion_matrix
cmn=confusion_matrix(y_test,y_pred)

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rmodel=RandomForestClassifier()
rmodel.fit(x_train,y_train)

y_pred=rmodel.predict(x_test)

from sklearn.metrics import confusion_matrix
cmr=confusion_matrix(y_test,y_pred)s

In [ ]:
##RNN
from keras.preprocessing import sequence
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence

embedding_vector_length=32
model = Sequential()

import numpy as np
np.random.seed(7)

x_train=sequence.pad_sequences(x_train,maxlen=80)
x_test=sequence.pad_sequences(x_test,maxlen=80)

model.add(Embedding(4736,200))

model.add(LSTM(200,dropout=0.2,recurrent_dropout=0.2))

model.add(Dense(1,activation='sigmoid'))

model.compile(loss='mean_absolute_error',optimizer='adam',metrics=['accuracy'])

model.summary()

model.fit(x_train,y_train,epochs=100,verbose=2,validation_data=(x_test,y_test))